In [9]:
import requests
import PyPDF2
import pandas as pd

import os
import re
import csv
import warnings
import pickle
warnings.filterwarnings("ignore") #Cert related warning


    
def Read_PDF(d, course_name):
    with open(ROOT_DIR+'/'+d+'/'+course_name, 'rb') as pdf_object:
        # First line is being read to check if it is a "file not found" html page
        temp = pdf_object.readline()
        temp = temp[:2]
        files_first_line = temp.decode()
        # in html page first two character would be <! and as the file is being
        # read as byte it is being converted to string by .decode()
        if files_first_line == '<!':
            pass
        else:
            pypdf2_object = PyPDF2.PdfFileReader(pdf_object)
            for page_no in range(0, pypdf2_object.numPages):
                page_object = pypdf2_object.getPage(page_no)
                total_text = page_object.extractText()
                r = re.compile(r'(\d{8})(\D+\s(\D*\s)*\D+)\d{2}(\d{10})')
                info = r.findall(total_text)
            return info
def Downloading_PDF(html_urls_list, pdf_urls_list):
    print('Downloading from PDFs Links')
    if not os.path.exists(ROOT_DIR):
        os.mkdir(ROOT_DIR)
    for html, pdfs in zip(html_urls_list, pdf_urls_list):
        folder_name = f"{ROOT_DIR}/{html.split('/')[-1]}"
        if not os.path.exists(folder_name):
            os.mkdir(folder_name)
        for pdf in pdfs:
            file_name = pdf.split('/')[-1]
            print(f"{folder_name}/{file_name}")
            if not os.path.exists(f'{folder_name}/{file_name}'):
                r = requests.get('http://'+pdf, stream=True, verify=CERT)
                chunk_size = 2000
                with open(f'{folder_name}/{file_name}', 'wb') as fd:
                    for chunk in r.iter_content(chunk_size):
                        fd.write(chunk)
                fd.close()
def Parsing_HTML_For_PDF_Links(urls):
    pattern = r'<a href="//(www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/[-a-zA-Z0-9]*/[\._,%+\sa-zA-Z0-9]*.pdf)" target="_blank"'

    print('Finding PDF links in these HTMLs')
    pdf_links = []
    for url in urls:
          #download the plain html file
        file = requests.get(url, timeout=20, verify=CERT)
        html = file.text
        link_pattern = re.compile(pattern)
        pdf_link = link_pattern.findall(html)
        if pdf_link == []:
            print(url)
        pdf_links.append(pdf_link)
    return pdf_links
def Parsing_PDF():
    if os.path.exists(STUDENT_INFO):
        print('Already Student Info Parsed')
        with open(STUDENT_INFO, 'rb') as f:
            db = pickle.load(f)
    else:
        print('Parsing Student Info')
        db = []
        dir_list = [d for d in os.listdir(ROOT_DIR) if not d.endswith('pkl')]
        for d in dir_list:
            file_list = os.listdir(ROOT_DIR+'/'+d)
            if not os.path.exists(f'{PICKLE_DIR}/{d}.pkl'):
                sem = []
                for course_name in file_list:
                    info = Read_PDF(d, course_name)
                    if info:
                        st_db = []
                        for i in info:
                            st_id, name, blank, reg = i
                            st_db.append((st_id, name, reg))
                        sem.append({ course_name: st_db })
                with open(f'{PICKLE_DIR}/{d}.pkl', 'wb') as f:
                    pickle.dump(sem, f)
                print(sem)
            else:
                with open(f'{PICKLE_DIR}/{d}.pkl', 'rb') as f:
                    sem = pickle.load(f)
                print(sem)
            db.append({ d : sem })
        with open(STUDENT_INFO, 'wb') as f:
            pickle.dump(db, f)
    return db

In [2]:
ROOT_DIR = 'data/pdf'
PICKLE_DIR = 'data/pickle'
SEAT_PLAN_HTML_LINKS = f'{PICKLE_DIR}/Seat_Plan_HTML_Links.pkl'
PDF_LINKS = f'{PICKLE_DIR}/PDF_Links.pkl'
STUDENT_INFO = f'{PICKLE_DIR}/Student_Info_Unstructured.pkl'
CSV_FILE = 'data/processed/student_info.csv'
CERT = False

In [3]:
root_url = "https://www.bracu.ac.bd/search/node/Final%20Examinations%20seat%20plan%20pdf"
root_pattern = re.compile(r'https://www.bracu.ac.bd/final-examinations-seat-plan-[-0-9a-z]+')
root_file = requests.get(root_url, timeout=20, verify=CERT)
secondary_url = root_pattern.findall(root_file.text)
secondary_url, len(secondary_url)

(['https://www.bracu.ac.bd/final-examinations-seat-plan-spring-2019',
  'https://www.bracu.ac.bd/final-examinations-seat-plan-summer-2019',
  'https://www.bracu.ac.bd/final-examinations-seat-plan-fall-2019',
  'https://www.bracu.ac.bd/final-examinations-seat-plan-fall-2018',
  'https://www.bracu.ac.bd/final-examinations-seat-plan-fall-2017',
  'https://www.bracu.ac.bd/final-examinations-seat-plan-spring-2017',
  'https://www.bracu.ac.bd/final-examinations-seat-plan-summer-2017',
  'https://www.bracu.ac.bd/final-examinations-seat-plan-spring-2018',
  'https://www.bracu.ac.bd/final-examinations-seat-plan-summer-2018'],
 9)

In [4]:
pdf_links = Parsing_HTML_For_PDF_Links(secondary_url)
pdf_links

Finding PDF links in these HTMLs


[['www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/Spring-2019/BCH202.pdf',
  'www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/Spring-2019/LAW201.pdf',
  'www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/Spring-2019/LAW303.pdf',
  'www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/Spring-2019/ARC122.pdf',
  'www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/Spring-2019/ARC123.pdf',
  'www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/Spring-2019/ARC224.pdf',
  'www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/Spring-2019/CSE101_0.pdf',
  'www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/Spring-2019/CSE230_0.pdf',
  'www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/Spring-2019/CSE320_0.pdf',
  'www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/Spring-2019/CSE421.pdf',
  'www.bracu.ac.bd/sites/default/files/registrar/exam_seat_plan/Spring-2019/ECO102_0.pdf',
  'www.bracu.

In [5]:
Downloading_PDF(secondary_url, pdf_links)

data/pdf/final-examinations-seat-plan-spring-2019/BCH202.pdf
data/pdf/final-examinations-seat-plan-spring-2019/LAW201.pdf
data/pdf/final-examinations-seat-plan-spring-2019/LAW303.pdf
data/pdf/final-examinations-seat-plan-spring-2019/ARC122.pdf
data/pdf/final-examinations-seat-plan-spring-2019/ARC123.pdf
data/pdf/final-examinations-seat-plan-spring-2019/ARC224.pdf
data/pdf/final-examinations-seat-plan-spring-2019/CSE101_0.pdf
data/pdf/final-examinations-seat-plan-spring-2019/CSE230_0.pdf
data/pdf/final-examinations-seat-plan-spring-2019/CSE320_0.pdf
data/pdf/final-examinations-seat-plan-spring-2019/CSE421.pdf
data/pdf/final-examinations-seat-plan-spring-2019/ECO102_0.pdf
data/pdf/final-examinations-seat-plan-spring-2019/ENG114_0.pdf
data/pdf/final-examinations-seat-plan-spring-2019/HST406_0.pdf
data/pdf/final-examinations-seat-plan-spring-2019/LAW344.pdf
data/pdf/final-examinations-seat-plan-spring-2019/cse220.pdf
data/pdf/final-examinations-seat-plan-spring-2019/cse221.pdf
data/pdf/fin

In [6]:
students = Parsing_PDF()

Parsing Student Info
[{'CSE419%2010AM.pdf': [('15101130', 'M. RAFIUL ALAM', '2056859270'), ('16101050', 'GAZI MUSA AL JOHAN', '5679262280'), ('18241017', 'FUAD HOSSAIN ONI', '0519471890'), ('16101243', 'SHIYAM TALUKDER', '2176836170'), ('16301217', 'MD. TAMJID HOSSAIN', '2184492620'), ('16301007', 'ANIK SARKER', '3599949350'), ('16301050', 'SHAFAITUL JANNAT', '8790670940'), ('16301122', 'FARHAN FUAD RUPOM', '2261978008'), ('16301131', 'MD. SHAMMYO SIKDER', '2110575520'), ('16301178', 'Md Omar Hasan', '1143384172'), ('16301188', 'KHANDAKAR RAZOAN AHMED', '8804239425'), ('14341015', 'MD. ZAFOR', '1976299762'), ('12201010', 'MD. TANZIR CHOWDHURY', '9850416221'), ('13101212', 'KAZI RIDWAN AREFIN', '7020732430'), ('17101009', 'SAYEEM MD. ABDULLAH', '1322774477'), ('17101045', 'HUMAIRA ISLAM', '2890017198'), ('17121014', 'MD. JUNAED IQBAL', '3935455352'), ('17101139', 'JIDNI MAYUKH', '7932108835'), ('17101297', 'HASNAT MD ABDULLAH', '2305136080'), ('17101304', 'MUTASIM RAFID', '1996185620'),

[{'CSE420.pdf': [('15301020', 'BOOSHRA NAZIFA MAHMUD', '8635282345'), ('15301029', 'S. M. A. MUKSIT CHOWDHURY', '6190928340'), ('15301033', 'ROISUL ISLAM RUMI', '3432343206'), ('15301040', 'TAMKIN MAHMUD TAN', '8852254078'), ('15301056', 'ANNA MARY MONDOL', '1818013752'), ('15301057', 'MOHAMED MOHIBUL HOSSAIN', '9432839427'), ('15301058', 'RAGHIB BARKAT MUHIB', '7580867693'), ('15301060', 'MD. IQBAL HOSSAIN', '0504565730'), ('15301061', 'FABIHA FAIROOZ', '5289990360'), ('15301062', 'ASHUB BIN ASAD', '1747939630'), ('15301064', 'SYED ZAFRUL HASSAN', '5621727890'), ('15301069', 'MD. MOSTAFIZUR RAHMAN', '2351314220'), ('15301077', 'NOSHIN NAWAL', '4681254910'), ('15301079', 'SABBIR AHMED', '0826436470'), ('15301087', 'NADIA FARHIN CHOWDHURY', '4018488900'), ('15301092', 'SYED MOAZZIM HOSSAIN', '3302437570'), ('15301105', 'SAMIN YEASAR SEAUM', '1857501729'), ('15301112', 'MEHADI HASAN HASIB', '1491353144'), ('15301119', 'AHMED SHAHRIAR', '1770106027'), ('15301120', 'SAQUEEB ABDULLAH', '601

[{'ECO101.pdf': [('17104137', 'SAMEA AHMAD', '9975006643'), ('17301169', 'MD. SAMIUL HAQUE', '3409721841'), ('17304102', 'AFIA MEHJABIN', '2707793524'), ('17301194', 'MD HASHIBUL ISLAM', '1910676924'), ('17301229', 'SHAILA SHARMIN', '6561535497'), ('18101563', 'MAHIMA RABBI', '7311153656'), ('18126071', 'SUMIYA AL SADIA MEGHA', '7318614476'), ('18101668', 'FARDEEN AHMED', '2793563861')]}, {'CSE260.pdf': [('17101231', 'SRIJON BISWAS', '6356747170'), ('17101234', 'SAFWAN RAHMAN', '8831861470'), ('17101261', 'MEHEDI HASSAN NAYEEM', '8136317831'), ('17101283', 'MISBAH UDDIN SAGOR', '1630788000'), ('17101304', 'MUTASIM RAFID', '4199618562'), ('17101306', 'MOHAMMED ZIAD HASSAN', '9408624035'), ('17101319', 'MAHARAB KIBRIA', '1554167698'), ('18301286', 'NISHAT SALSABIL', '3333968537'), ('17101369', 'FAZLE RABBI FAIYAZ', '3981532280'), ('17101379', 'MD.SABBIR AHMED', '3053269618'), ('17101381', 'SPANDAN SARKAR', '7292161912'), ('17101393', 'ABU TAHIR NIZAM', '3307043551'), ('17101396', 'SAMIA 

[{'CSE420.pdf': [('14101180', 'MOHAMMED HASIN ISHRAK', '0141185610'), ('14301032', 'MD. ISHAN AREFIN HOSSAIN', '3072318990'), ('14301059', 'FABIAN PARSIA GEORGE', '3447618200'), ('14301064', 'MRIDUL BANIK', '3314890470'), ('14301132', 'MD. SOWAD HASSAN', '4485263546'), ('14301133', 'ISMAIL HOSSAIN', '4605553101'), ('14301136', 'MD. YOUSUF HOSSAIN', '8457261221'), ('12201022', 'BM ABIR', '2922728612'), ('12201035', 'MD. SAJIBUR RAHMAN SARKER', '2742394797'), ('12201037', 'SIDDIQ HUSAIN TASHFEEN', '3829878848'), ('12201038', 'SALEM QUDDUS FAHAD', '9575025121'), ('12201104', 'PREMA DEV', '8676129592'), ('12201106', 'MOSTOFA SAIF REZA', '4628725799'), ('12201111', 'KHAN RAFIN AHMED', '3080857229'), ('12301002', 'TONURUHA AMEEN', '6790054983'), ('12301013', 'S.M. TAREQUL HASAN ROBIN', '8812094980'), ('15241003', 'ZINNATUN NESHA', '3662036220'), ('12301043', 'MD TAWABUL ISLAM', '3472555390'), ('12301047', 'SAMRIN SULTANA TITHI', '9124545140'), ('13101035', 'KAMRUN NAHAR RUMA', '5134453400'),

[{'CSE420.pdf': [('14101156', 'RIASAT ISLAM KHAN', '8968183310'), ('14201046', 'FARHAN SHAHRIAR', '2775597380'), ('16101170', 'SALMAN SAMI KHAN', '6633732090'), ('16241009', 'MEHREEN KHAN', '0082130850'), ('16101292', 'JOYASISH MAZUMDER', '8794675580'), ('16301068', 'SAMIUL ISLAM SHAD', '5159986990'), ('17201150', 'ALIMUL HASAN JAMI', '7469198380'), ('16301216', 'SHADMAN SHAHRIAR HASAN ARKO', '9289370810'), ('17301174', 'SUMSIL ARAFIN PRANTA', '8631744200'), ('16201020', 'MASHRUF AHMED ZAFRI', '5898285848'), ('19341012', 'ISHMAM AHMED SOLAIMAN', '4644278940'), ('17101020', 'MALIHA MONAMI', '4837988229'), ('17121014', 'MD. JUNAED IQBAL', '3935455352'), ('17101137', 'MD. FARHAN ZAMAN', '4001189504'), ('17101153', 'SHAKIB MAHMUD DIPTO', '3258467153'), ('17101185', 'NISRAT JAHAN', '2184954280'), ('17109028', 'MD. MONTASIR NAFIZ', '6188511460'), ('17101235', 'MOHAMMAD SHEHABUL ISLAM', '3398783030'), ('17101240', 'AFIA FAHMIDA RAHMAN', '6092945610'), ('17101343', 'SHAHNEWAZ ALI MOHAMMAD', '6

[{'CSE420.pdf': [('14301098', 'MD. RAKIBUL ISLAM', '7293558670'), ('14301120', 'AHMED NAHIAN', '7326255970'), ('14201020', 'JAHID HASAN MAMUN', '1807759600'), ('15101021', 'KISHWAR MAHEEN', '5691559934'), ('15101037', 'PRINCE ELVIS GOMES', '5120884587'), ('15101060', 'ABDULLA AL REZA', '2113968399'), ('15101073', 'FARDEEN AHMED', '7605365830'), ('15301012', 'ARNAB BARUA', '5291885171'), ('15301023', 'MD. ABU TAHAMID', '7821069699'), ('15301046', 'AFIA ANJUM PREETY', '5363151865'), ('15341004', 'KAMONASHISH SAHA', '7304132707'), ('13301107', 'SYED MARUF HOSSAIN', '6017307849'), ('15201032', 'MD JUBAYER', '4033210580'), ('16141001', 'MD HASIB ULLAH BHUIYAN', '4160291450'), ('16101016', 'NAHIAN KABIR', '8421345140'), ('16101026', 'MD.MUYEED SHAHRIAR', '8284090700'), ('16101027', 'FAHIM AHMED', '3942054920'), ('16101035', 'G. M. MAHBUBUR RAHMAN', '9574154970'), ('16101045', 'MOHAMMAD WASEQ-UL ISLAM', '1126043210'), ('16101145', 'FAHIM ISHRAK', '2485176680'), ('16101193', 'RIDWANA TASNIM', 

[{'CSE420.pdf': [('13101118', 'RASHIDUL HASAN JUNAYED', '8268663348'), ('16341020', 'RAFEE MIZAN KHAN CHOWDHURY', '9140731760'), ('13101215', 'A M M GOLAM YEAZDANI', '1657871066'), ('13101252', 'SAJJAD HOSSAIN', '3547911965'), ('13301043', 'NIGAR SULTANA', '2831605476'), ('13301055', 'UMMAY SANI PRIA', '3944371258'), ('13301114', 'TAWSIFUL ALAM', '9604251159'), ('13301118', 'YASIN AHMED', '7372984176'), ('13101244', 'NAHEEN HOSSAIN', '8816342664'), ('13321003', 'AAREEF ABRAR', '7784462144'), ('13201074', 'SHOAIB HOSSAIN', '2123309869'), ('14301005', 'BIR BALLAV ROY', '8230841470'), ('14301017', 'MD. SHAIKH ASHIKUL ALAM', '6079530710'), ('14301019', 'ABDULLAHIL KAFI', '4681124020'), ('14301052', 'TASNIA NOBI AFEE', '8760053610'), ('14301063', 'SAMIHA TASNIM', '4621735170'), ('14301065', 'RAMISA IBNAT MORSHED', '8401270730'), ('14301104', 'MD. MAZIDUL HASAN', '5116721990'), ('14341010', 'MD. RIDOWAN CHOWDHURY', '7875080770'), ('11101018', 'MD SAMIUL BASHAR AUVEE', '7512569350'), ('112010

[{'CSE320_0.pdf': [('17201056', 'JABID HASSAN', '3430677706'), ('17241013', 'KAZI AHNAF ALAVEE', '7279896019'), ('17241016', 'SYED SAKIB SALIM MAHMUD', '4265049128'), ('17201124', 'ZEBEL-E-NOOR AKHAND', '7592521524'), ('17201140', 'SAADMAN AHMED', '8339076035'), ('17201146', 'RIASAT MAHBUB', '6926565180'), ('18101202', 'MD.SOLAIMAN HOSSAIN', '1938756139'), ('18101556', 'ASIF ATIQ', '2902074230')]}, {'PHY102.pdf': [('14208016', 'FARIHA RAHMAN', '7842338530'), ('16108002', 'EREM EHSAN RISHAD', '4396256730'), ('16308016', 'J M RAKAYET', '2089090860'), ('13308007', 'RAIHATUL ZANNAH MIMMA', '1564444510'), ('16208001', 'PANTHA SHAHRIAR', '2166425550'), ('16208002', 'WAEZA RAFIQUE SALWA', '3865915390'), ('16208003', 'SOHANA KHAN', '8457424590'), ('16208004', 'MEHRAN HAIDER', '9536347530'), ('16208007', 'ADHORA IFRIT WAHIDA', '7566394028'), ('16208008', 'SHANILA ISLAM', '6718819513'), ('17108003', 'AKIBUR RAHMAN', '5908051048'), ('18108024', 'PARISA FAIZA', '1157475822'), ('18108002', 'FAKIR A

In [7]:
infos = []
for student in students:
    for exam in student.keys():
        exam_name = exam[len("final-examinations-seat-plan-"):].upper()
        #print(exam_name)
        for pages in student[exam]:
            for subject in pages.keys():
                for st in pages[subject]:
                    brac_id = st[0].strip()
                    #if brac_id does not starts with 1. Then some parsing problem has happend
                    if brac_id.startswith('1'):
                        name = st[1].strip().upper()
                        reg = st[2].strip()
                        subject_name = subject.replace("%20","").split('.')[0][:6].upper()
                        info = [name, brac_id, reg, subject_name, exam_name]
                        infos.append(info)
infos

[['M. RAFIUL ALAM', '15101130', '2056859270', 'CSE419', 'FALL-2019'],
 ['GAZI MUSA AL JOHAN', '16101050', '5679262280', 'CSE419', 'FALL-2019'],
 ['FUAD HOSSAIN ONI', '18241017', '0519471890', 'CSE419', 'FALL-2019'],
 ['SHIYAM TALUKDER', '16101243', '2176836170', 'CSE419', 'FALL-2019'],
 ['MD. TAMJID HOSSAIN', '16301217', '2184492620', 'CSE419', 'FALL-2019'],
 ['ANIK SARKER', '16301007', '3599949350', 'CSE419', 'FALL-2019'],
 ['SHAFAITUL JANNAT', '16301050', '8790670940', 'CSE419', 'FALL-2019'],
 ['FARHAN FUAD RUPOM', '16301122', '2261978008', 'CSE419', 'FALL-2019'],
 ['MD. SHAMMYO SIKDER', '16301131', '2110575520', 'CSE419', 'FALL-2019'],
 ['MD OMAR HASAN', '16301178', '1143384172', 'CSE419', 'FALL-2019'],
 ['KHANDAKAR RAZOAN AHMED', '16301188', '8804239425', 'CSE419', 'FALL-2019'],
 ['MD. ZAFOR', '14341015', '1976299762', 'CSE419', 'FALL-2019'],
 ['MD. TANZIR CHOWDHURY', '12201010', '9850416221', 'CSE419', 'FALL-2019'],
 ['KAZI RIDWAN AREFIN', '13101212', '7020732430', 'CSE419', 'FALL

In [10]:
header = ['Name', 'ID', 'Reg. No.', 'Course', 'Semester']

with open(CSV_FILE, 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(infos)

df = pd.read_csv(CSV_FILE)
df

,Name,ID,Reg. No.,Course,Semester
0,M. RAFIUL ALAM,15101130,2056859270,CSE419,FALL-2019
1,GAZI MUSA AL JOHAN,16101050,5679262280,CSE419,FALL-2019
2,FUAD HOSSAIN ONI,18241017,519471890,CSE419,FALL-2019
3,SHIYAM TALUKDER,16101243,2176836170,CSE419,FALL-2019
4,MD. TAMJID HOSSAIN,16301217,2184492620,CSE419,FALL-2019
...,...,...,...,...,...
19332,MOSTOFA KAMAL SAGOR,17301106,8021713730,CSE330,SPRING-2019
19333,MOENUL HASAN,17301119,3556928555,CSE330,SPRING-2019
19334,MD. AZMOL FUAD,17301154,5351838825,CSE330,SPRING-2019
19335,ATHAR FAISAL,17305016,6062715488,CSE330,SPRING-2019


In [11]:
df['Course'].sort_values().unique()

array(['ACT201', 'ACT202', 'ACT301', 'ACT422', 'ACT425', 'ACT431',
       'ANT101', 'ANT102', 'ANT103', 'ANT104', 'ANT355', 'ARC121',
       'ARC122', 'ARC123', 'ARC224', 'ARC225', 'ARC231', 'ARC232',
       'ARC241', 'ARC242', 'ARC251', 'ARC252', 'ARC293', 'ARC326',
       'ARC327', 'ARC331', 'ARC343', 'ARC393', 'ARC431', 'ARC432',
       'ARC441', 'ARC452', 'ARC493', 'ARC494', 'ARC522', 'BCH101',
       'BCH102', 'BCH201', 'BCH202', 'BIO101', 'BTE101', 'BTE102',
       'BTE103', 'BTE201', 'BTE202', 'BTE203', 'BTE204', 'BTE302',
       'BTE303', 'BTE304', 'BTE306', 'BTE307', 'BTE308', 'BTE317',
       'BTE401', 'BTE402', 'BTE403', 'BTE405', 'BTE411', 'BTE417',
       'BUS101', 'BUS201', 'BUS202', 'BUS203', 'BUS301', 'BUS302',
       'BUS321', 'CEE211', 'CEE212', 'CEE213', 'CEE311', 'CEE312',
       'CEE411', 'CHE101', 'CHE110', 'CHN101', 'CSE101', 'CSE110',
       'CSE111', 'CSE112', 'CSE161', 'CSE220', 'CSE221', 'CSE230',
       'CSE250', 'CSE251', 'CSE260', 'CSE310', 'CSE320', 'CSE3

In [12]:
#ID and Reg. No. are wrong. For some of it, it was just parsed wrong but 
df[df['ID'].apply(str).apply(lambda x: x[0]) != '1']

,Name,ID,Reg. No.,Course,Semester


In [13]:
df['ID'].sort_values().unique()

array([10105011, 10110006, 10110007, ..., 19346060, 19346062, 19346064])

In [14]:
total_number_student_info = len(df['ID'].unique())
total_number_student_info

8452